## DATA TRANSFORMATION

**2025-10-07**

In [1]:
import pandas as pd
import numpy as np
import pickle
from pathlib import Path

Importamos el CSV

In [2]:
input_path = Path("data") / "staySpain_cleaned.pkl"
df = pd.read_pickle(input_path)

Creamos una copia del DF para hacer los cambios

In [3]:
df_transf = df.copy()

### Price per person

In [4]:
posicion = 13
df_transf.insert(posicion, 'pricexperson', df_transf['price'] / df_transf['accommodates'])

### Ocupation and Ocupation%

Transformación de columnas availabilty a positivo + porcentaje

In [5]:
posicion = 18
df_transf.insert(posicion, 'ocupation30', 30 - df_transf['availability_30'])

In [6]:
posicion = 19
df_transf.insert(posicion, 'ocup%30', (df_transf['ocupation30'] / 30) * 100)

Vamos a trabajar con la columna de disponibilidad mensual ya que el KPI se evalúa mensualmente, por lo que aunque estén creadas las columnas de porcentaje de tasa de ocupación según las diferentes variables temporales, no las vamos a utilizar.

In [7]:
posicion = 21
df_transf.insert(posicion, 'ocupation60', 60 - df_transf['availability_60'])

In [8]:
#df['ocup60%'] = (df['ocupation60'] / 60) * 100

In [9]:
posicion = 23  # Cambia según tu necesidad
df_transf.insert(posicion, 'ocupation90', 90 - df_transf['availability_90'])

In [10]:
#df['ocup90%'] = (df['ocupation90'] / 90) * 100

In [11]:
posicion = 25  # Cambia según tu necesidad
df_transf.insert(posicion, 'ocupation365', 365 - df_transf['availability_365'])

In [12]:
#df['ocup365%'] = (df['ocupation365'] / 365) * 100

### Satisfacción general

Creación de columna índice de satisfacción general

In [13]:
general_satisf = {
    'accuracy':      df_transf.review_scores_accuracy.mean(),
    'cleanliness':   df_transf.review_scores_cleanliness.mean(),
    'checkin':       df_transf.review_scores_checkin.mean(),
    'communication': df_transf.review_scores_communication.mean(),
    'location' :     df_transf.review_scores_location.mean()}

sorted_avg_scores = sorted(general_satisf.items(), key=lambda item: item[1], reverse=True)

df_transf['general_satisf'] = df_transf[
    ['review_scores_accuracy', 'review_scores_cleanliness', 
     'review_scores_checkin', 'review_scores_communication', 
     'review_scores_location']
].mean(axis=1)

posicion = 36
col_general_satisf = df_transf.pop('general_satisf')
df_transf.insert(posicion, 'general_satisf', col_general_satisf)

### Amenities_score - optional: score for each amenities group

Creación de la columna amenities score

In [14]:
# Define amenities for each category/group
# Groups are made in relation to the aspects of an accomodation customers find more important based on sector research
amenities_ac = ['heating', 'heated floors', 'air conditioning']
amenities_outdoor = ['balcony', 'terrace', 'backyard', 'patio', 'outdoor dining area', 'outdoor seating']
amenities_parking = ['parking']
amenities_view = ['beach view', 'beachfront', 'waterfront', 'lake', 'mountain view']
amenities_internet = ['ethernet', 'wifi', 'internet']
amenities_work = ['workspace', 'mbps', 'office']
amenities_kitchen = ['dishwasher', 'refrigerator', 'freezer', 'washer', 'kitchen', 'stove', 'oven', 'toaster', 'full kitchen', 'microwave', 'kettle']
amenities_bathroom = ['essentials', 'tub', 'shower', 'bathtub', 'hair dryer']
amenities_sport = ['gym', 'pool']
amenities_disabled = ['wheelchair', 'hoist', 'single level', 'wide hallways', 'wide doorway', 'grab bars', 'shower chair', 'wide clearance', 'elevator', 'accessible', 'bath chair']


# Some groups have multiple small items why other are some but big items
# i.e. lots of appliances to make a kitchen, but not so many to make a parking
# Define scores for each amenities list based on the amount and importance of items
amenities_scores = {
    'amenities_ac': (amenities_ac, 5.0), 
    'amenities_outdoor': (amenities_outdoor, 5.0),   
    'amenities_parking': (amenities_parking, 10.0),   
    'amenities_view': (amenities_view, 10.0),
    'amenities_internet': (amenities_internet, 10.0),
    'amenities_work': (amenities_work, 10.0),
    'amenities_kitchen': (amenities_kitchen, 2.0),
    'amenities_bathroom': (amenities_bathroom, 3.0),
    'amenities_sport': (amenities_sport, 10.0),
    'amenities_disabled': (amenities_disabled, 2.0)
}


# Function to count occurrences with type checking and lowercase conversion
def count_amenities(amenities_string, amenities, score):
    if isinstance(amenities_string, str):  # Check if the cell is a string
        amenities_string = amenities_string.lower()  # Convert to lowercase
        total_score = sum(score for amenity in amenities if amenity in amenities_string)
        return min(total_score, 10)  # Cap the score at 10
    else:
        return np.nan  # Return NaN for non-string values


# Iterate through each amenities list and create new columns with score for each one
for key, (amenities, score) in amenities_scores.items():
    df_transf[f'{key}_score'] = df_transf['amenities_list'].apply(lambda x: count_amenities(x, amenities, score))


# The following columns are related to amenities score
amenities_score_columns = [
    'amenities_ac_score', 
    'amenities_outdoor_score',   
    'amenities_parking_score',   
    'amenities_view_score',
    'amenities_internet_score',
    'amenities_work_score',
    'amenities_kitchen_score',
    'amenities_bathroom_score',
    'amenities_sport_score',
    'amenities_disabled_score'
]


# Based on the previous lists, a total score of amenities is calculated
df_transf['amenities_score'] = df_transf[amenities_score_columns].sum(axis=1)

# To keep the dataframe simple, detailed group scores are removed
df_transf = df_transf.drop(columns=amenities_score_columns)

### Zona

La columna zona se calcula a partir de las columnas "city", "neighbourhood_district", "neighbourhood_name".

No existe un criterio unificado en el dataset original, por lo que se busca resumirlas en el mayor nivel de granularidad disponible.

Separadas con una barra baja, existen las siguientes opciones:
- Ciudad_Distrito_Barrio
- Ciudad_Barrio (Para ciudades que no indican distritos)
- Ciudad_Población

In [15]:
# Concatenate with fillna
df_transf['zona'] = (
    df_transf['city'].fillna('') + '_' + 
    df_transf['neighbourhood_district'].fillna('') + '_' + 
    df_transf['neighbourhood_name'].fillna('')
)

# Then clean up consecutive underscores and trailing/leading underscores
df_transf['zona'] = df_transf['zona'].str.replace(r'_+', '_', regex=True).str.strip('_')

### Data transfomed overview

In [16]:
df_transf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9650 entries, 0 to 9999
Data columns (total 44 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   apartment_id                 9650 non-null   int64         
 1   name                         9650 non-null   object        
 2   description                  9650 non-null   object        
 3   host_id                      9650 non-null   int64         
 4   neighbourhood_name           9650 non-null   object        
 5   neighbourhood_district       5860 non-null   object        
 6   room_type                    9650 non-null   object        
 7   accommodates                 9650 non-null   int64         
 8   bathrooms                    9576 non-null   Int64         
 9   bedrooms                     9581 non-null   Int64         
 10  beds                         9605 non-null   Int64         
 11  amenities_list               9634 non-null   obj

In [17]:
df_transf.head()

,apartment_id,name,description,host_id,neighbourhood_name,neighbourhood_district,room_type,accommodates,bathrooms,bedrooms,...,review_scores_location,review_scores_value,general_satisf,reviews_per_month,country,city,insert_date,is_instant_bookable,amenities_score,zona
0,11964,A Room With A View,Private Bedroom In Our Attic Apartment Right I...,45553,Centro,NaN,Private Room,2,2,1,...,10.0,10.0,10.0,75.0,Spain,Malaga,2018-07-31,False,40.0,Malaga_Centro
1,21853,Bright And Airy Room,We Have A Quiet And Sunny Room With A Good Vie...,83531,Crmenes,Latina,Private Room,1,1,1,...,8.0,9.0,9.2,52.0,Spain,Madrid,2020-01-10,False,58.0,Madrid_Latina_Crmenes
2,32347,Explore Cultural Sights From A Family-Friendly...,Open French Doors And Step Onto A Plant-Filled...,139939,San Vicente,Casco Antiguo,Entire Home Apt,4,1,2,...,10.0,10.0,10.0,142.0,Spain,Sevilla,2019-07-29,True,48.0,Sevilla_Casco Antiguo_San Vicente
3,35379,Double 02 Casanovarooms Barcelona,Room At A My Apartment Kitchen And 2 Bathrooms...,152232,L'Antiga Esquerra De L'Eixample,Eixample,Private Room,2,2,1,...,10.0,9.0,9.8,306.0,Spain,Barcelona,2020-01-10,True,56.0,Barcelona_Eixample_L'Antiga Esquerra De L'Eixa...
4,35801,Can Torras Farmhouse Studio Suite,Lay In Bed Watch Sunlight Change The Mood Of D...,153805,Quart,NaN,Private Room,5,1,2,...,10.0,10.0,10.0,39.0,Spain,Girona,2019-02-19,False,66.0,Girona_Quart


In [18]:
output_path = r"data/staySpain_transformed.pkl"
df_transf.to_pickle(output_path)
output_path2 = r"data/staySpain_transformed.csv"
df_transf.to_csv(output_path2)
print(f"Datos limpios guardados en: {output_path}")
print(f"Datos limpios guardados en: {output_path2}")

Datos limpios guardados en: data/staySpain_transformed.pkl
Datos limpios guardados en: data/staySpain_transformed.csv
